## **Import the relevant libraries**

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Data

In [2]:
# Loading the mnist datasets and its informations
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

# Preprocessing-creating train, validation, test and scaling
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# scaling using the highest possible value 255
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

# Applying custom transformation
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

# Shuffling
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# Batching
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Outline the model

In [3]:
# Set the sized of the inputs, output and hidden layers
input_size = 784
output_size = 10
hidden_layer_size = 50

# Define how the model will look like
model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
])

C:\Users\HP\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Choose the optimizer and the loss function

In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training

In [5]:
# Determine the number of epochs
NUM_EPOCHS = 5

# Fit the model
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 18s - 34ms/step - accuracy: 0.8855 - loss: 0.4050 - val_accuracy: 0.9375 - val_loss: 0.2090
Epoch 2/5
540/540 - 7s - 13ms/step - accuracy: 0.9475 - loss: 0.1817 - val_accuracy: 0.9542 - val_loss: 0.1567
Epoch 3/5
540/540 - 7s - 13ms/step - accuracy: 0.9579 - loss: 0.1394 - val_accuracy: 0.9622 - val_loss: 0.1275
Epoch 4/5
540/540 - 7s - 13ms/step - accuracy: 0.9671 - loss: 0.1108 - val_accuracy: 0.9642 - val_loss: 0.1164
Epoch 5/5
540/540 - 7s - 12ms/step - accuracy: 0.9710 - loss: 0.0953 - val_accuracy: 0.9697 - val_loss: 0.0973


## Test the model

In [6]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.9696 - loss: 0.1010


In [8]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}'.format(test_loss, test_accuracy*100.))

Test loss: 0.10. Test accuracy: 96.96
